In [ ]:
# Testing optimization of constructLineRecursive with many tests

def getPlay(dominoes, myHand, available):
    handValues = dominoes[myHand]
    lineOptions = np.full((len(available), len(dominoes)), False)
    for idx, value in enumerate(available):
        idxPlayable = np.where(np.any(handValues==value, axis=1))[0]
        lineOptions[idx, myHand[idxPlayable]] = True
    
    locations, dominoes = np.where(lineOptions) # find where options are available
    if len(locations)>0:
        playidx = np.random.choice(len(locations),1)[0]
        return locations[playidx], dominoes[playidx]
    return None, None

def uniqueSequences(lineSequence, lineDirection, updatedLine):
    seen = set()
    uqSequence = []
    uqDirection = []
    uqUpdated = []
    for subSeq, subDir, subUpdate in zip(lineSequence, lineDirection, updatedLine):
        subSeqTuple = tuple(subSeq)
        if subSeqTuple not in seen:
            seen.add(subSeqTuple)
            uqSequence.append(subSeq)
            uqDirection.append(subDir)
            uqUpdated.append(subUpdate)
    return uqSequence, uqDirection, uqUpdated

def updateLine(lineSequence, lineDirection, nextPlay, onOwn, updatePlayIdx):
    if nextPlay is None: return lineSequence, lineDirection
    if lineSequence==[[]]: return lineSequence, lineDirection
    
    newSequence, newDirection, updatedLine = [], [], []
    if onOwn:
        for pl,dr in zip(lineSequence,lineDirection):
            if pl[0]==nextPlay:
                newSequence.append([updatePlayIdx[pp] for pp in pl[1:]])
                newDirection.append(dr[1:])
                updatedLine.append(True)
    else:
        for pl,dr in zip(lineSequence,lineDirection):
            if nextPlay in pl: 
                idxInLine = np.where(pl==nextPlay)[0][0]
                if idxInLine>0:
                    newSequence.append([updatePlayIdx[pp] for pp in pl[:idxInLine]])
                    newDirection.append(dr[:idxInLine])
                    updatedLine.append(True)
            else:
                newSequence.append([updatePlayIdx[pp] for pp in pl])
                newDirection.append(dr)
                updatedLine.append(False)
    
    uqSequence, uqDirection, uqUpdated = uniqueSequences(newSequence, newDirection, updatedLine)
    
    if uqSequence==[]: return [[]], [[]]
    
    subsumed = [False]*len(uqSequence)
    for idx, (seq, updated) in enumerate(zip(uqSequence, uqUpdated)):
        if updated:
            for icmp, scmp in enumerate(uqSequence):
                if len(scmp)>len(seq):
                    if seq==scmp[:len(seq)]:
                        subsumed[idx]=True
                        continue
    
    finalSequence = [uqSeq for (uqSeq, sub) in zip(uqSequence, subsumed) if not(sub)]
    finalDirection = [uqDir for (uqDir, sub) in zip(uqDirection, subsumed) if not(sub)]
    return finalSequence, finalDirection


def compareSequences(numCompare=100, printOutput=False):
    if printOutput: numCompare=1
    timeStandard = 0
    timeUpdate = 0
    
    allGood = True
    for _ in range(numCompare):
        # start by creating dominoes
        highestDominoe = 9
        dominoes = df.listDominoes(highestDominoe)
        numDominoes = len(dominoes)

        # Then, put some dominoes in the players hand
        numInHand = int(numDominoes/4)
        myHand = np.random.permutation(numDominoes)[:numInHand]

        # Define a game availability
        numPlayers = 4
        available = np.random.randint(0, highestDominoe+1, numPlayers)

        lineSequence, lineDirection = df.constructLineRecursive(dominoes[myHand], available[0])

        loc,dom = getPlay(dominoes, myHand, available)
        if loc is not None:
            playIdx = np.where(myHand==dom)[0][0]
            playDirection, nextAvailable = df.playDirection(available[loc], dominoes[dom])
            newAvailable = copy(available)
            newAvailable[loc] = nextAvailable
        else:
            playIdx = None
            newAvailable = copy(available)

        newHand = np.delete(myHand, np.arange(len(myHand))==playIdx)    
        
        tupd = time.time()
        if loc is not None:
            playIdx = np.where(myHand==dom)[0][0]
        updatePlayIdx = dict(zip(range(len(myHand)), np.cumsum(np.arange(len(myHand))!=playIdx)-1)) 
        newSequence, newDirection = updateLine(lineSequence, lineDirection, playIdx, loc==0, updatePlayIdx)
        timeUpdate += time.time() - tupd
        
        tsta = time.time()
        newSeq, newDir = df.constructLineRecursive(dominoes[newHand], newAvailable[0])
        timeStandard += time.time() - tsta
            
        allGood &= (newSeq==newSequence)
        allGood &= (newDir==newDirection)
        if not(allGood):
            printOutput=True
            
        if printOutput:
            print("Hand: ")
            print(dominoes[myHand])
            print("")
            
            print(f"Available values: {available}\n")
            print("Line sequences: ")
            df.gameSequenceToString(dominoes[myHand], lineSequence, lineDirection)
            print("")
            
            if loc is not None:
                print(f"The play is dominoe: {dom} ({df.dominoesString(dominoes[dom])}) on line: {loc} (handIdx: {playIdx}).")
                print(f"New available: {newAvailable}")
                print("")
                
            print("New hand: ")
            print(dominoes[newHand])
            print("")
            
            print(f"Update Line Method: ")
            df.gameSequenceToString(dominoes[newHand], newSequence, newDirection)
            print("")

            print(f"Construct Line Recursive after play: ")
            df.gameSequenceToString(dominoes[newHand], newSeq, newDir)
            print("")
            
            print(f"Update Line Method: ")
            print(newSequence)
            
            print(f"Construct line method: ")
            print(newSeq)
            
        if not(allGood):
            return allGood, timeStandard, timeUpdate
            
    return allGood, timeStandard, timeUpdate

outcome, timeStandard, timeUpdate = compareSequences(numCompare=10000, printOutput=False)
print(f"All equal: {outcome}. Time standard: {timeStandard}. Time udpate: {timeUpdate}")